## Submitting a job with the [`otello`](https://github.com/hysds/otello) python library

#### Once your job-type has been registered and built (see: [pge_create.ipynb](pge_create.ipynb)), jobs can be submitted from python using the steps laid out in this notebook.

#### While this notebook only shows submission of a single job/parameter-set, you can map or iterate over a collection of input parameter sets to efficiently submit large batches of jobs.

### Establish an otello `Mozart` instance to communicate with the HySDS cluster controller.
#### It will be necessary to provide credentials the first time you initialise otello.

#### When prompted for the HySDS host, include the protocol, e.g. https://my-mozart.jpl.nasa.gov

#### When prompted for "HySDS cluster authenticated", enter 'y' if the cluster requires a password to access.

In [ ]:
import json
import os
import otello
import re
import shutil

from pathlib import Path
from pprint import pprint

if not os.path.exists(f"{Path.home()}/.config/otello/config.yml"):
    otello.client.initialize()

m = otello.mozart.Mozart()

### Instantiating a `JobType` object

#### *You will need to ensure that the the tag (i.e. the portion following the colon) matches the branch name used by your repository.*

#### *You will also need to replace `<YOUR_JOB_TYPE_NAME>` with the name of your job's notebook (without the filetype extension)*
#### e.g. if your notebook is hello_world_sample_pge.ipynb, the hysds-io.json and job-spec.jsons will have suffix `hello_world_sample_pge` and the value of `job_type` below would be `job-hello_world_sample_pge:develop` or similar.

In [ ]:
job_type_name = 'job-<YOUR_JOB_TYPE_NAME>:develop'  # This will need to be customised using the relevant action name (ie job/hysds-spec suffix) and branch name.

job_type = m.get_job_types()[job_type_name]
job_type.initialize()

### Getting useful information about the job-type

These commands list (respectively)
- the available queues
- the input schema (parameters) of the job-type
- the default arguments for the job-type

In [ ]:
job_type.get_queues()
print(job_type.describe())
pprint(job_type.get_input_params())

### Specifying arguments to pass when running the job

#### Here is where user-defined parameter values are specified, making sure to remain consistent with value types as indicated in the *helloworld_notebook*. Default values are used where none are provided (as we've done here with *start_orbit_number*). *set_input_params* is called to pass the parameter values to the job.

In [ ]:
custom_parameters = {
    'str_arg': 'THIS IS SINGLE JOB 1'
}

job_type.set_input_params(custom_parameters)
pprint(job_type.get_input_params())

### Submitting the job
#### A job tag (useful for finding the job later) and job queue are specified. Both are optional. Job submission is asynchronous, so this call will return almost immediately.

In [ ]:
from datetime import datetime

sample_job_tag = f'{datetime.strftime(datetime.now(), "%Y%m%d")}_single_submission_job_test'
print(sample_job_tag)

job_run = job_type.submit_job(tag=sample_job_tag, queue="factotum-job_worker-small")

### Determining job completion
#### Information about the job state will print periodically, until the job is completed.

In [ ]:
job_run.wait_for_completion()

### Submitting Multiple Jobs: Specifying input argument sets
#### It is possible to iterate through a list of input argument sets and submit many jobs asynchronously.  Jobs will be processed in parallel using the PCM cluster's compute node fleet.

In [ ]:
common_tag = f'{datetime.now().strftime("%Y%m%d")}multiple_submission_job_test'
input_parameter_sets = [{
    'tags': f'{common_tag}_1',
    'params': {
        'str_arg': 'THIS IS MULTIPLE JOB 1'
    }
},
    {
        'tags': f'{common_tag}_2',
        'params': {
            'str_arg': 'THIS IS MULTIPLE JOB 2'
        }
    }
]

### Submitting Multiple Jobs: Submitting the Jobs 
Now submit the jobs and keep jobs in otello's job_set data structure

In [ ]:
job_set = otello.JobSet()

for parameter_set in input_parameter_sets:
    job_type.set_input_params(parameter_set["params"])
    job = job_type.submit_job(tag=parameter_set["tags"], queue="factotum-job_worker-small")
    job_set.append(job)

### Determining completion of the job-set
#### Information about the jobs' states will print periodically, until all the jobs are completed.

In [ ]:
job_set.wait_for_completion()

### Getting metadata for a job-set

#### This metadata includes AWS S3 bucket paths where the ingested data is located

In [ ]:

products_metadata = []
for job in job_set:
    try:
        product_metadata = job.get_generated_products()
        print(json.dumps(products_metadata, indent=2, sort_keys=True))
        products_metadata.append(product_metadata)
    except Exception as e:
        print(e)

### Authenticate to AWS

#### In a terminal window, run `aws-login -p default` and enter your credentials

### Download the jobs' data products to a local directory

In [ ]:
product_directories = []
for product_metadata in products_metadata:
    try:
        s3_product_url = re.sub(r'^s3://.+?/(.+)$', r's3://\1', product_metadata[0]['urls'][-1])

        destination_directory = os.path.basename(s3_product_url)

        if os.path.isdir(destination_directory):
            shutil.rmtree(destination_directory)

        print(f'Running "aws s3 sync {s3_product_url} {destination_directory}"')
        !aws s3 sync $s3_product_url $destination_directory

        product_directories.append(destination_directory)
    except Exception as e:
        print(e)

for product_directory in product_directories:
    !ls $product_directory